In [ ]:
!pip install xgboost

### IMPORTAR LIBREARÍAS

In [7]:
import pandas as pd
import pymysql
import random 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

random.seed(42)

### Generar Dataframe a partir de tablas

In [8]:
conn = pymysql.connect(host='localhost', user='tfm_user', password='tfmdatabase1234', db='tfm_database')

inmueble_query = "SELECT * FROM inmueble"
anuncio_query = "SELECT * FROM anuncio"

df_inmueble = pd.read_sql(inmueble_query, conn)
df_anuncio = pd.read_sql(anuncio_query, conn)

inmueble_columns = ["id_inmueble", "exterior", "num_habitaciones", "superficie", "ascensor", "parking"]
anuncio_columns = ["id_anuncio", "id_inmueble", "precio"]

df_inmueble = df_inmueble[inmueble_columns]
df_anuncio = df_anuncio[anuncio_columns]
df = df_anuncio.merge(df_inmueble, how="left", on="id_inmueble")

### Transformar el Dataframe para ser entrenado

In [9]:
df_machine = pd.get_dummies(df)
x_column = list(set(df_machine.columns) - set(["precio", "id_anuncio", "id_inmueble"]))
y_column = ["precio"]
X = df_machine[x_column]
y = df_machine[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.fit_transform(X_test)


MODELO SIMPLE

In [10]:
model = RandomForestRegressor()
model.fit(X_train_scale, y_train)

c:\Users\aaira\OneDrive\Documentos\tfm_eae\.venv\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor()

In [11]:
y_train_predict = model.predict(X_train_scale)
y_test_predict = model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

0.7374348960637581

ESCALAR VARIABLE TARGET

In [ ]:
y_train_scale = scaler.fit_transform(y_train)
y_test_scale = scaler.fit_transform(y_test)

In [ ]:
model = RandomForestRegressor()
model.fit(X_train_scale, y_train_scale)

In [ ]:
y_train_predict = model.predict(X_train_scale)
y_test_predict = model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

AJUSTE DE PARÁMETROS

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 5, 10],
    'max_features':['log2', 'sqrt', 'auto']
}

In [ ]:
model_gridcv = GridSearchCV(model, params, cv=3)
model_gridcv.fit(X_train_scale, y_train_scale)
best_model = model_gridcv.best_estimator_

In [ ]:
y_train_predict = best_model.predict(X_train_scale)
y_test_predict = best_model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))

OTRO MODELO

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model = XGBRegressor()

# Define the parameter grid for grid search
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Perform grid search with cross-validation
xgb_model_gridcv = GridSearchCV(xgb_model, params, cv=3)
xgb_model_gridcv.fit(X, y)
best_xgb_model = xgb_model_gridcv.best_estimator_

In [ ]:
y_train_predict = best_xgb_model.predict(X_train_scale)
y_test_predict = best_xgb_model.predict(X_test_scale)

print("Train Accuracy: ", r2_score(y_train, y_train_predict))
print("Test Accuracy: ", r2_score(y_test, y_test_predict))

print("MSE Train: ", mean_squared_error(y_train, y_train_predict))
print("MSE Test: ", mean_squared_error(y_test, y_test_predict))